In [1]:
import pymongo
from pymongo import MongoClient
from gensim import corpora, models, similarities
from gensim.models import Word2Vec as w2v
from collections import defaultdict
import jieba
import io
import pymongo
from pymongo import MongoClient
import json
import jieba
from gensim import corpora, models, similarities
from collections import defaultdict
import jieba
import io

stoplist = []
documents =[]
words=[]
client = MongoClient() #預設連接自己主機上的mongodb
database = client["test"] # SQL: Database Name
collection = database["test"]
jieba.set_dictionary('dict1.txt')
jieba.load_userdict("dict_keyw_new.txt") 
jieba.load_userdict("dict_cbdic.txt") 
with io.open('C:/Users/BIG DATA/Desktop/stoplist.txt','r',encoding = 'utf-8') as f:
    for line in f.readlines():
        stoplist.append(line.strip('\n'))
for post in collection.find():
    documents.append(post['content'])

for doc in range(0,len(documents)):
    word=[]
    z=unicode(documents[doc])
    for a in jieba.cut(z):
        if not a in stoplist:
            
            ab=[a]
            word+=ab
    #print word
    words+=[word]
model = w2v(words, min_count=1, size=400)

Building prefix dict from C:\Users\BIG DATA\dict1.txt ...
Dumping model to file cache c:\users\bigdat~1\appdata\local\temp\jieba.ueb82094ea19eedbe1ac558ddc779bc67.cache
Loading model cost 1.947 seconds.
Prefix dict has been built succesfully.


In [5]:
similar=model.most_similar(u'大象',topn=30)
for a in similar:
    print a[0],a[1]

長頸鹿 0.761223077774
烏龜 0.755196869373
兔子 0.741585671902
企鵝 0.734549701214
黑熊 0.730184555054
牠們 0.728037655354
北極熊 0.720403969288
猩猩 0.71906208992
小動物 0.718579411507
袋鼠 0.716046273708
鱷魚 0.709764897823
兩隻 0.698158979416
鳥兒 0.695076465607
黑猩猩 0.68273550272
牠 0.682170629501
獼猴 0.681883871555
幼鳥 0.679917752743
海豹 0.676669955254
無尾熊 0.676360070705
蛇 0.674084246159
這隻 0.67353194952
蟒蛇 0.673329293728
紅毛猩猩 0.673172652721
浣熊 0.671310961246
鸚鵡 0.670680463314
蜜蜂 0.666467905045
松鼠 0.665662050247
貓熊 0.66475456953
貓 0.663328230381
蝙蝠 0.661493241787


In [6]:
def W2VClassification(classes,corpus,similarLevel):
    dicT = {}   #類別字典
    Expt = []   #無法被W2V辨識的
    Remain = [] #低於門檻值的
    cnt = [] #計算長度用的陣列
    
    for a in classes:  #把類別名稱,當作Key,加入字典
        dicT[a] = set() #把每個Key的Value設為set ( 避免重複 )
        
    for q in open(corpus,'r'):   #把萃取關鍵字的結果讀出來
        q = q.strip().decode('utf-8')   #去空白,解碼
        cnt.append(q)  #把每一行都加進去-->用來算長度
        x1 = []
        try:
            for u in classes:  #從自訂好的類別一個個去跟'每個關鍵字'比對相似度
                x1.append(float(model.similarity(q,u.decode('utf-8')))) #把相似度寫進x1理面
            for i in range(0,len(classes)): #從 1~ 類別群數量
                if x1[i] == sorted(x1,reverse = True)[0]:  #後者:排序過後的第一個(相似度最高的);
                                                           #前者:遍歷每一個x1
                                                           #也就是問,這個關鍵字跟哪個類別最像
                    if x1[i] >= similarLevel: #設定相似門檻,過濾相似度過低的
                        dicT[classes[i]].add(q) #把相似度最高的加進字典的該類別裡
                    else:
                        Remain.append(q) #沒過門檻值的
        except:
            Expt.append(q) #W2V不認識的
            
    #把原長度cnt,跟去重複長度(轉成set)都印出來
    print '原總量',len(cnt),'\t|','去重複後',len(set(cnt))
    print '--------------------------------------------------'
    
    #把分好類的每一個類別內容印出來
    Scnt = 0
    for w in dicT:
        print '【',w,'】','\t數量:',len(dicT[w])
        for i in dicT[w]:
            print i,
            Scnt += 1
        print '\n'

    print '--------------------------------------------------'
    print '相似度門檻',similarLevel,'\t|','成功分類',Scnt,'\t|','成功分類比率',int(float(Scnt)/float(len(set(cnt)))*100),'%'
    print '--------------------------------------------------\n'

    #在此之前Expt 和 Remain都是List , 都有照順序排好 , 印出來可檢查這些詞的原順序
    #此時把List轉成Set , 去除重複值 , 增加可讀性
    Expt = set(Expt)
    Remain = set(Remain)
    
    #把W2V不認識的印出來
    print '【W2V無法辨識】\t數量',len(Expt)
    for r in Expt:
        print r,
    print '\n'
    
    #把沒過門檻值的印出來
    print '【低於門檻值】\t數量',len(Remain)
    for r in Remain:
        print r,
    print '\n'

In [7]:
classes = ['國際','社會','體育','政治','生活','股票','蔡英文','林書豪']
W2VClassification(classes,'dict_keyw_new.txt',0.2)

原總量 95656 	| 去重複後 95436
--------------------------------------------------
【 蔡英文 】 	數量: 1909
立委候選人 吳育昇 三點 邱靖雅 鍾易仲 立委蔡錦隆 行政院版 紀念儀式 國科會主委 民意論壇 潘懷宗 何煖軒 議會黨團 底牌 國父遺像 國旗 新閣員 卡式台胞證 航空城 鄧志賢 世代傳承 宣明智 八年前 中央政府 閩南語 質詢 顏清標 一中各表 大陸方面 民間版 潘翰疆 鄉親 退選 示範區 投書 亞太再平衡 瘦肉精 溝通 舉手 中央黨部 陳以信 行程安排 歐巴馬 松機 訪大陸 新閣 台南市長 關中 林萬億 蔡錦隆 主桌 海外僑胞 長陳菊 甜粄 講清楚 公聽會 蕭美琴 馬政府 陳銘薰 黨版 張榮 當選人 母雞 釣魚台 郭正亮 王義川 民進黨立委 謝俊雄 造勢晚會 核四封存 柯陣營 政治行情 電視辯論 藍皮書 大台中 意願書 桃園航空城 楊偉中 老柯 美帝 青年政策 徐世榮 段宜康 李世光 伯恩斯 完全執政 台大校友 選情告急 李桐豪 國務院 重畫 鄭成功祖廟 陳柏志 接軌 請益 協調會議 楊瓊瓔 台灣海峽 台灣燈會 鄭正鈐 吳秀明 包可士 因應方案 廖萬隆 冷處理 嚴家淦 元首 換位 連戰此行 黃適卓 石化專區 國民黨要 國軍 兩區 陳鴻禧 鄭金隆 施金樹 黃重球 黨團 歐巴馬總統 內閣總辭 特偵組 憲兵 張肇良 各黨 證嚴法師 許淑華 重啟談判 雙橡園 親民黨團 廢核大遊行 習會 萬人送仲丘 藍營 李志鏞 向歐巴馬 友邦 林光華 黨團大會 台聯黨 黃宏斌 和平甜柿 領表 陳冲 馬吳 徵詢過 監察院 記者會中 入閣 黨部 有再 駐美代表處 換人做 高孔廉 台聯黨團 黨主席 餐敘 麻生 催票 應曉薇 兩岸經貿 金銀三會 立委選舉 善意 羅紹和 幹事長 簡太郎 反核團體 不分區 兩岸協議 台海 請大家 陳致中 宋楚瑜 彭淮南 杜紫軍 大老 九二共識 主管長官 南部辦公室 蔡玉玲 國民黨主席 財政部高層 王金平 政見會 邱鏡淳 賴儀松 請問 卜睿哲 國機國造 蘇麗瓊 連勝文 鐵路地下化 劉政鴻 李根政 李翔宙 孔傑榮 周禮良 宋嫌 施政報告 沈富雄 雙城論壇 林啟滄 黃建中 反核人士 轉述 臨時會 林德旺 黃昆輝 余陳月瑛 民進黨要 加強溝通 母喪 專家團 張國鑫 三組 小英 王世堅 邱志